# Train Classifier

## Libraries

In [1]:
import os, json
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # or any {'0', '1', '2','3'}
from shutil import rmtree


from tensorflow import keras
from keras import applications
from keras import callbacks


##  Definitions

In [2]:
v_num_of_classes=30 #not needed
v_input_shape=(224, 224, 3)
v_path_dataset='../data/sampledataset'
v_image_size=(224, 224)
v_color_mode="rgb" # "rgb", "rgba", or "grayscale"
v_batch_size=16 
v_epochs = 10

v_metrics=['MeanSquaredError','AUC','Precision','Recall','accuracy']
v_workspace='./exp_Xception'


## Functions

In [3]:
# Dataset
def load_dataset(root_dataset='../data/sampledataset', image_size=(299,299),color_mode='grayscale', batch_size=16 ):
    datagen = keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=True,
        featurewise_std_normalization=False,
        samplewise_std_normalization=True,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        brightness_range=None,
        shear_range=0.0,
        zoom_range=(0.95,0.95),#####
        channel_shift_range=0.0,
        fill_mode='nearest',
        cval=0.0,
        horizontal_flip=False,
        vertical_flip=False,
        rescale=1./255,#######
        preprocessing_function=None,
        data_format=None,# 'channels_last'
        validation_split=0.2,
        dtype=None,#tf.float32
    )

    ds_train = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='training',
        interpolation='nearest'
    )

    ds_validate = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='validation',
        interpolation='nearest',
    )

    #class_names=ds_train.class_indices
    #print (class_names)
    #class_names = list(class_names.keys())
    return (ds_train, ds_train.labels),(ds_validate,ds_validate.labels)

In [4]:
def save_class_indices(class_indices_json,path_to_workspace):
    file_path=os.path.join(path_to_workspace,'class_indices.json')
    class_indices_json = dict((v,k) for k,v in class_indices_json.items())
    print (class_indices_json)
    with open(file_path, "w") as write_file:
        json.dump(class_indices_json, write_file)

In [5]:
rmtree(v_workspace) if os.path.isdir(v_workspace) else None
os.mkdir(v_workspace) 

#os.mkdir(v_workspace) if not(os.path.isdir(v_workspace)) else rmtree(v_workspace)

path_csvname = os.path.join(v_workspace,'performance.csv')
path_tensorboardLog= os.path.join(v_workspace,'tensorboard.csv')
path_checkpoint=os.path.join(v_workspace,'checkpoint')

                            
CSVLogger_cb = callbacks.CSVLogger(path_csvname,  separator=',',   append=True)


TensorBoard_cb = callbacks.TensorBoard(log_dir=path_tensorboardLog,
                                          histogram_freq=1,
                                          write_graph=True,
                                          write_images=True,
                                          update_freq=v_batch_size,
                                          #write_steps_per_second=False,
                                          profile_batch=2,
                                          embeddings_metadata=None)


ModelCheckpoint_cb = callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                save_best_only=True, ###to save space
                                                save_weights_only=False,
                                                monitor='val_accuracy',
                                                mode='max')


EarlyStopping_cb = callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=0.001,
                                                   patience=6,
                                                   verbose=0, 
                                                   mode='auto',
                                                   baseline=None,
                                                   restore_best_weights=False)

TerminateOnNaN_cb = callbacks.TerminateOnNaN()

ReduceLROnPlateau_cb = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                            factor=0.01,
                                                            patience=5,
                                                            verbose=0,
                                                            mode='auto',
                                                            min_delta=0.001,
                                                            cooldown=0,
                                                            min_lr=0)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * tf.math.exp(-0.0001)

LearningRateScheduler_cb = callbacks.LearningRateScheduler(scheduler, 
                                                                    verbose=0)



RemoteMonitor_cb = callbacks.RemoteMonitor(root='http://localhost:9000',
                                                    path='/publish/epoch/end/',
                                                    field='data',
                                                    headers=None,
                                                    send_as_json=False)

## Modeling

In [6]:
# https://lightrun.com/answers/keisen-tf-keras-vis-graph-disconected--transfer-learning-model-with--top-layer
def get_model():
    
    '''
    applications.inception_v3.InceptionV3
    applications.vgg16.VGG16
    applications.xception.Xception
    applications.mobilenet_v2.MobileNetV2

    '''
    base_model = applications.xception.Xception(
        weights='imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=v_input_shape,
        include_top=False) 
    base_model.trainable = False
    
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(v_num_of_classes, activation='softmax')(x)
    outputs= keras.layers.Softmax()(x)
    model = keras.Model(base_model.inputs, outputs)
    return model
model=get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

## Data

In [7]:
(x_train, y_train), (x_valid, y_valid)=load_dataset(root_dataset=v_path_dataset, 
                                                    image_size=v_image_size,
                                                    color_mode=v_color_mode, 
                                                    batch_size=v_batch_size )
num_of_classes=x_train.num_classes
class_indices=x_train.class_indices

Found 680 images belonging to 30 classes.
Found 156 images belonging to 30 classes.


In [8]:
save_class_indices(class_indices_json=class_indices,path_to_workspace=v_workspace)

{0: 'Cherry', 1: 'Coffee-plant', 2: 'Cucumber', 3: 'Fox_nut(Makhana)', 4: 'Lemon', 5: 'Olive-tree', 6: 'Pearl_millet(bajra)', 7: 'Tobacco-plant', 8: 'almond', 9: 'banana', 10: 'cardamom', 11: 'chilli', 12: 'clove', 13: 'coconut', 14: 'cotton', 15: 'gram', 16: 'jowar', 17: 'jute', 18: 'maize', 19: 'mustard-oil', 20: 'papaya', 21: 'pineapple', 22: 'rice', 23: 'soyabean', 24: 'sugarcane', 25: 'sunflower', 26: 'tea', 27: 'tomato', 28: 'vigna-radiati(Mung)', 29: 'wheat'}


## Training

In [9]:
# compile
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=v_metrics
              )

In [10]:
#Train the top layer
callbacksList=[CSVLogger_cb,TensorBoard_cb,ModelCheckpoint_cb,EarlyStopping_cb,TerminateOnNaN_cb,ReduceLROnPlateau_cb]
history=model.fit(x=x_train,
                  validation_data=(x_valid),
                  batch_size=v_batch_size,
                  epochs=v_epochs, 
                  callbacks=callbacksList,
                  shuffle=True
                 )


Epoch 1/10
43/43 [==============================] - 82s 2s/step - loss: 3.3296 - mean_squared_error: 0.0320 - auc: 0.6100 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.1765 - val_loss: 3.2225 - val_mean_squared_error: 0.0317 - val_auc: 0.7298 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 2/10
43/43 [==============================] - 75s 2s/step - loss: 3.0741 - mean_squared_error: 0.0312 - auc: 0.8207 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.5338 - val_loss: 3.0676 - val_mean_squared_error: 0.0312 - val_auc: 0.8105 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.4872 - lr: 0.0010
Epoch 3/10
43/43 [==============================] - 82s 2s/step - loss: 2.9116 - mean_squared_error: 0.0306 - auc: 0.8801 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.6529 - val_loss: 3.0053 - val_mean_squared_error: 0.0309 - val_auc: 0.8426 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 -

## Save

In [11]:
fname = os.path.join (v_workspace,"saved_model")
savedModel = os.path.join(v_workspace, "saved_model")
savedWeights = os.path.join(v_workspace,"saved_weights","weights")

model.save(savedModel)
model.save_weights(savedWeights)

INFO:tensorflow:Assets written to: ./exp_Xception/saved_model/assets
